**1** Install dependencies in Colab

In [2]:
import os
from tqdm import tqdm


In [3]:
html_output_dir = "ocr_visualization_pages"
os.makedirs(html_output_dir, exist_ok=True)


In [8]:
!apt-get update
!apt-get install -y unrar tesseract-ocr


Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://cli.github.com/packages stable/main amd64 Packages [345 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,229 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,863 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu

**🔹 2. Install Tigrinya & Amharic traineddata**

In [9]:
!apt-get install -y tesseract-ocr-amh tesseract-ocr-tir


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tesseract-ocr-amh tesseract-ocr-tir
0 upgraded, 2 newly installed, 0 to remove and 73 not upgraded.
Need to get 2,153 kB of archives.
After this operation, 5,878 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-amh all 1:4.00~git30-7274cfa-1.1 [1,856 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-tir all 1:4.00~git30-7274cfa-1.1 [297 kB]
Fetched 2,153 kB in 2s (1,158 kB/s)
Selecting previously unselected package tesseract-ocr-amh.
(Reading database ... 121689 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-amh_1%3a4.00~git30-7274cfa-1.1_all.deb ...
Unpacking tesseract-ocr-amh (1:4.00~git30-7274cfa-1.1) ...
Selecting previously unselected package tesseract-ocr-tir.
Preparing to unpack .../tesseract-ocr-tir_1%3a4.00

In [4]:
!rm -r ocr_visualization_pages


In [5]:
!ls

sample_data


In [10]:
!tesseract --list-langs


List of available languages (4):
amh
eng
osd
tir


**🔹 3. Install Python libraries**

In [11]:
!pip install pytesseract pandas tqdm jiwer


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 52.9 MB/s eta 0:00:00


🔹 4. Upload and extract ocr_eval.**rar**

In [12]:
from google.colab import files
import os

print("Upload your data.rar file")
uploaded = files.upload()

# Extract RAR
!unrar x ocr_eval.rar


Upload your data.rar file


Saving ocr_eval.rar to ocr_eval.rar

UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from ocr_eval.rar

Creating    ocr_eval                                                  OK
Creating    ocr_eval/gt                                               OK
Extracting  ocr_eval/gt/page_001.txt                                       0%  OK 
Extracting  ocr_eval/gt/page_002.txt                                       0%  OK 
Extracting  ocr_eval/gt/page_003.txt                                       0%  OK 
Extracting  ocr_eval/gt/page_004.txt                                       0%  OK 
Extracting  ocr_eval/gt/page_005.txt                                       0%  OK 
Extracting  ocr_eval/gt/page_006.txt                                       0%  OK 
Extracting  ocr_eval/gt/page_007.txt                                       0%  OK 
Extracting  ocr_eval/gt/page_008.txt                                

🔹 5. Import libraries & define **paths**

In [14]:
import os
import pytesseract
import pandas as pd
from tqdm import tqdm
from jiwer import wer


In [15]:
image_dir = "ocr_eval/images"
gt_dir = "ocr_eval/gt"


🔹 6. OCR + utility **functions** **bold text**

In [16]:
def run_ocr(image_path, lang="tir"):
    return pytesseract.image_to_string(image_path, lang=lang).strip()
def load_gt(txt_path):
    with open(txt_path, "r", encoding="utf-8") as f:
        return f.read().strip()


In [17]:
def image_to_base64(img_path):
    with open(img_path, "rb") as f:
        encoded = base64.b64encode(f.read()).decode("utf-8")
    ext = os.path.splitext(img_path)[1].lower().replace(".", "")
    return f"data:image/{ext};base64,{encoded}"


**7. CER (Character Error Rate)**

In [18]:
def cer(ref, hyp):
    import editdistance
    return editdistance.eval(ref, hyp) / max(len(ref), 1)


In [19]:
!pip install editdistance


**8. Normalize text (important!)**

In [20]:
def normalize_text(text):
    return " ".join(text.split())


Helper: Character-Level Alignment & Highlighting

In [21]:
from difflib import SequenceMatcher
import html

def highlight_char_errors(gt_text, ocr_text):
    """
    Returns HTML string with character-level OCR errors highlighted.
    """
    sm = SequenceMatcher(None, gt_text, ocr_text)
    highlighted = []

    for tag, i1, i2, j1, j2 in sm.get_opcodes():
        if tag == "equal":
            highlighted.append(html.escape(ocr_text[j1:j2]))

        elif tag == "replace":  # substitution
            for ch in ocr_text[j1:j2]:
                highlighted.append(
                    f'<span class="substitution">{html.escape(ch)}</span>'
                )

        elif tag == "insert":  # insertion
            for ch in ocr_text[j1:j2]:
                highlighted.append(
                    f'<span class="insertion">{html.escape(ch)}</span>'
                )

        elif tag == "delete":  # deletion (show placeholder)
            for _ in gt_text[i1:i2]:
                highlighted.append(
                    '<span class="deletion">␣</span>'
                )

    return "".join(highlighted)


In [22]:
import numpy as np
from difflib import SequenceMatcher

def levenshtein_stats(ref, hyp, level="char"):
    """
    Compute substitutions, deletions, insertions, and error positions.

    level: "char" or "word"
    """
    if level == "word":
        ref = ref.split()
        hyp = hyp.split()

    sm = SequenceMatcher(None, ref, hyp)

    subs, dels, ins = 0, 0, 0
    error_positions = []

    for tag, i1, i2, j1, j2 in sm.get_opcodes():
        if tag == "replace":
            subs += max(i2 - i1, j2 - j1)
            error_positions.append({
                "type": "substitution",
                "ref_pos": (i1, i2),
                "ocr_pos": (j1, j2)
            })
        elif tag == "delete":
            dels += (i2 - i1)
            error_positions.append({
                "type": "deletion",
                "ref_pos": (i1, i2),
                "ocr_pos": None
            })
        elif tag == "insert":
            ins += (j2 - j1)
            error_positions.append({
                "type": "insertion",
                "ref_pos": None,
                "ocr_pos": (j1, j2)
            })

    return subs, dels, ins, error_positions


def char_accuracy(ref, hyp):
    if len(ref) == 0:
        return 0.0
    return 1.0 - cer(ref, hyp)


def word_accuracy(ref, hyp):
    ref_words = ref.split()
    hyp_words = hyp.split()
    if len(ref_words) == 0:
        return 0.0
    return 1.0 - wer(ref, hyp)


 Run OCR on 10 images and evaluate

In [23]:
results = []

image_list = sorted(os.listdir(image_dir))[:110]

for img_name in tqdm(image_list):
    img_path = os.path.join(image_dir, img_name)

    base_name = os.path.splitext(img_name)[0]
    gt_path = os.path.join(gt_dir, base_name + ".txt")

    if not os.path.exists(gt_path):
        print(f"⚠️ Missing GT for {img_name}, skipping.")
        continue

    ocr_text = run_ocr(img_path, lang="tir")
    gt_text = load_gt(gt_path)

    # Character-level stats
    c_sub, c_del, c_ins, c_err_pos = levenshtein_stats(gt_text, ocr_text, level="char")

    # Word-level stats (normalized)
    gt_norm = normalize_text(gt_text)
    ocr_norm = normalize_text(ocr_text)

    w_sub, w_del, w_ins, w_err_pos = levenshtein_stats(gt_norm, ocr_norm, level="word")

    results.append({
        "Image": img_name,

        # Standard Metrics
        "CER": cer(gt_text, ocr_text),
        "WER": wer(gt_norm, ocr_norm),

        # Accuracy Metrics
        "Char_Accuracy": char_accuracy(gt_text, ocr_text),
        "Word_Accuracy": word_accuracy(gt_norm, ocr_norm),

        # Character-level Errors
        "Char_Substitutions": c_sub,
        "Char_Deletions": c_del,
        "Char_Insertions": c_ins,

        # Word-level Errors
        "Word_Substitutions": w_sub,
        "Word_Deletions": w_del,
        "Word_Insertions": w_ins,

        # Detailed Error Positions (for analysis)
        "Char_Error_Positions": c_err_pos,
        "Word_Error_Positions": w_err_pos
    })


100%|██████████| 110/110 [05:17<00:00,  2.89s/it]


In [24]:
import base64

one downloadable HTML file per image.

In [25]:

import zipfile
html_output_dir = "ocr_error_highlight_pages"
os.makedirs(html_output_dir, exist_ok=True)

for img_name in tqdm(image_list):
    img_path = os.path.join(image_dir, img_name)
    base_name = os.path.splitext(img_name)[0]

    gt_path = os.path.join(gt_dir, base_name + ".txt")
    if not os.path.exists(gt_path):
        continue

    # Load data
    gt_text = load_gt(gt_path)
    ocr_text = run_ocr(img_path, lang="tir")

    highlighted_text = highlight_char_errors(gt_text, ocr_text)

    # Convert image to Base64
    img_base64 = image_to_base64(img_path)

    html_page = f"""
<!DOCTYPE html>
<html>
<head>
<meta charset="UTF-8">
<title>OCR Error Visualization – {img_name}</title>

<style>
body {{
    font-family: Arial, sans-serif;
    margin: 30px;
}}

.container {{
    display: flex;
    gap: 30px;
    align-items: flex-start;
}}

.image-box, .text-box {{
    flex: 1;
}}

.image-box img {{
    max-width: 100%;
    border: 1px solid #aaa;
}}

pre {{
    white-space: pre-wrap;
    font-family: "Courier New", monospace;
    font-size: 14px;
    padding: 15px;
    border: 1px solid #aaa;
    background: #f9f9f9;
}}

.substitution {{ background-color: #ffb3b3; }}
.insertion {{ background-color: #b3d9ff; text-decoration: underline; }}
.deletion {{ background-color: #dddddd; color: #555; }}

.legend span {{
    padding: 4px 8px;
    margin-right: 10px;
    border-radius: 4px;
    font-size: 13px;
}}
</style>
</head>

<body>

<h1>OCR Character-Level Error Visualization</h1>

<div class="legend">
    <span class="substitution">Substitution</span>
    <span class="insertion">Insertion</span>
    <span class="deletion">Deletion</span>
</div>

<br>

<div class="container">

    <div class="image-box">
        <h2>Input Image</h2>
        <img src="{img_base64}">
    </div>

    <div class="text-box">
        <h2>Tesseract OCR Output (Errors Highlighted)</h2>
        <pre>{highlighted_text}</pre>
    </div>

</div>

</body>
</html>
"""

    html_path = os.path.join(html_output_dir, base_name + ".html")
    with open(html_path, "w", encoding="utf-8") as f:
        f.write(html_page)

print(f"✅ HTML pages generated successfully in: {html_output_dir}")


100%|██████████| 110/110 [05:09<00:00,  2.81s/it]

✅ HTML pages generated successfully in: ocr_error_highlight_pages


In [27]:
zip_path = "ocr_error_highlight_pages.zip"

with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zipf:
    for root, _, files_in_dir in os.walk(html_output_dir):
        for file in files_in_dir:
            full_path = os.path.join(root, file)
            zipf.write(full_path, arcname=file)

print("📦 ZIP file created:", zip_path)

files.download(zip_path)


📦 ZIP file created: ocr_error_highlight_pages.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [26]:
df = pd.DataFrame(results)
display(df.drop(columns=["Char_Error_Positions", "Word_Error_Positions"]))

print("\n=== Overall Average Metrics ===")
print("Average CER:", df["CER"].mean())
print("Average WER:", df["WER"].mean())
print("Average Character Accuracy:", df["Char_Accuracy"].mean())
print("Average Word Accuracy:", df["Word_Accuracy"].mean())

print("\n=== Total Error Counts (Character Level) ===")
print("Substitutions:", df["Char_Substitutions"].sum())
print("Deletions:", df["Char_Deletions"].sum())
print("Insertions:", df["Char_Insertions"].sum())

print("\n=== Total Error Counts (Word Level) ===")
print("Substitutions:", df["Word_Substitutions"].sum())
print("Deletions:", df["Word_Deletions"].sum())
print("Insertions:", df["Word_Insertions"].sum())


,Image,CER,WER,Char_Accuracy,Word_Accuracy,Char_Substitutions,Char_Deletions,Char_Insertions,Word_Substitutions,Word_Deletions,Word_Insertions
0,page_001.png,0.045156,0.109170,0.954844,0.890830,58,18,4,28,0,0
1,page_002.png,0.052914,0.136929,0.947086,0.863071,51,24,4,32,0,1
2,page_003.png,0.013158,0.064516,0.986842,0.935484,2,0,0,2,0,0
3,page_004.png,0.036900,0.129213,0.963100,0.870787,25,1,4,23,0,0
4,page_005.png,0.069645,0.141026,0.930355,0.858974,85,7,0,22,0,0
...,...,...,...,...,...,...,...,...,...,...,...
105,page_106.png,0.039409,0.189873,0.960591,0.810127,17,0,2,14,0,1
106,page_107.png,0.105145,0.285000,0.894855,0.715000,95,1,22,55,0,2
107,page_108.png,0.177520,0.475771,0.822480,0.524229,271,15,23,111,0,1
108,page_109.png,0.166939,0.475000,0.833061,0.525000,80,0,40,49,0,8



=== Overall Average Metrics ===
Average CER: 0.2023359974633703
Average WER: 0.35262398585976523
Average Character Accuracy: 0.7976640025366297
Average Word Accuracy: 0.647376014140235

=== Total Error Counts (Character Level) ===
Substitutions: 26115
Deletions: 5916
Insertions: 4438

=== Total Error Counts (Word Level) ===
Substitutions: 6562
Deletions: 217
Insertions: 153
